# Setup

## Paquetes

In [126]:
# !pyenv local meyf-kaggle-02
# !pyenv shell dmeyf-kaggle-02
# !pyenv versions
# !pyenv virtualenv 3.8.7 dmeyf-kaggle-02
#!pyenv version



In [127]:
# %%bash 
# pip install duckdb
# pip install jupysql
# pip install duckdb-engine

In [128]:
# %%bash 
# pip install pandas

## BD


In [145]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:////Users/mkiszkurno/Documents/dmeyf/datasets/dmeyf.db


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Y ya podemos usar **SQL** dentro de una notebook!

## Creacion Tabla

In [130]:
%%time
%%sql
create or replace table competencia_02 as
select
    *
from read_csv_auto('/Users/mkiszkurno/Documents/dmeyf/datasets/competencia_02_crudo.csv.gz')

CPU times: user 3min 4s, sys: 1min 30s, total: 4min 35s
Wall time: 2min 8s


,Success


## Chequeos
Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [146]:
%%time
%sql SELECT count(*) FROM COMPETENCIA_02

CPU times: user 131 ms, sys: 210 ms, total: 341 ms
Wall time: 520 ms


,count_star()
0,4562810


In [147]:
%%time

%sql SELECT * FROM COMPETENCIA_02 LIMIT 12

CPU times: user 195 ms, sys: 593 ms, total: 788 ms
Wall time: 692 ms


,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,8.0,0.00,-35773.32,2.33,5148.0,36602.13,54.0,0.0,5008.71,CONTINUA
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,...,1.0,0.00,-67414.82,1149.57,3299.0,28503.06,10.0,0.0,1395.87,CONTINUA
2,29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,...,1.0,0.00,0.00,0.00,6790.0,5170.25,2.0,0.0,0.00,CONTINUA
3,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,22.0,69653.59,-59422.00,0.00,7037.0,67991.90,31.0,0.0,2897.31,CONTINUA
4,29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,...,8.0,0.00,-4802.21,0.00,7596.0,4851.93,3.0,0.0,492.66,CONTINUA
5,29188277,201901,1,0,1,65,296,1672.70,16113.26,796.59,...,1.0,0.00,-5791.17,0.00,7827.0,12539.39,6.0,0.0,797.64,CONTINUA
6,29188490,201901,1,0,1,43,281,-520.87,6116.69,2138.32,...,8.0,9172.86,-29810.71,0.00,7568.0,73132.65,41.0,0.0,9172.86,CONTINUA
7,29188883,201901,1,0,1,64,296,64862.09,459723.16,988.28,...,8.0,0.00,-1332.76,0.00,7504.0,1102.50,1.0,0.0,58.65,CONTINUA
8,29189950,201901,1,0,1,56,240,13535.41,41250.39,7158.16,...,1.0,0.00,-20419.25,66.25,3717.0,27701.56,19.0,0.0,1137.81,CONTINUA
9,29190599,201901,1,0,1,59,296,2244.16,24670.53,113.76,...,22.0,8727.12,-16718.98,37.05,7137.0,1388.73,2.0,0.0,1219.92,CONTINUA


In [148]:
%%time
%%sql
SELECT foto_mes, 
       COUNT(*) as cantidad 
  FROM competencia_02
 GROUP BY foto_mes

CPU times: user 189 ms, sys: 43.4 ms, total: 232 ms
Wall time: 342 ms


,foto_mes,cantidad
0,201901,124864
1,201902,125898
2,201903,126536
3,201904,127059
4,201905,127683
5,201906,129181
6,201907,130551
7,201908,132615
8,201909,134267
9,201910,136515


In [134]:
%%time
%%sql
SELECT numero_de_cliente, 
       COUNT(*) as cantidad 
  FROM competencia_02
 GROUP BY numero_de_cliente
 having cantidad between 5 and 15
 order by cantidad desc

CPU times: user 2.42 s, sys: 437 ms, total: 2.86 s
Wall time: 572 ms


,numero_de_cliente,cantidad
0,154893503,15
1,55268609,15
2,59950254,15
3,64971122,15
4,82360772,15
...,...,...
27139,91513950,5
27140,93751779,5
27141,93810421,5
27142,93971634,5


# Creacion Clase Ternaria

In [135]:
%%time
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes 
      from competencia_02
), clientes as (
    select distinct numero_de_cliente 
     from competencia_02
), todo as (
    select numero_de_cliente, foto_mes 
     from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , 
        CASE
            WHEN mes_0 + mes_1 + mes_2 = 1 THEN 'BAJA+1'
            WHEN mes_0 + mes_1 + mes_2 = 2 THEN 'BAJA+2'
            WHEN mes_0 + mes_1 + mes_2 = 3 THEN 'CONTINUA'
        END AS clase_ternaria
    from todo t
    left join competencia_02 c using (numero_de_cliente, foto_mes)
) select
  foto_mes
  , numero_de_cliente
  , clase_ternaria,
  mes_0,
  mes_1,
  mes_2
from clase_ternaria where mes_0 = 1

CPU times: user 12 s, sys: 2.29 s, total: 14.3 s
Wall time: 2.04 s


,Success


In [136]:
%%time
%%sql 

SELECT * FROM targets T
--where T.clase_ternaria IS NOT NULL 
--where T.numero_de_cliente = 29187499
--where T.numero_de_cliente = 29185587
--where T.numero_de_cliente = 105799389
where T.numero_de_cliente = 118785515
	


CPU times: user 139 ms, sys: 19.4 ms, total: 159 ms
Wall time: 326 ms


,foto_mes,numero_de_cliente,clase_ternaria,mes_0,mes_1,mes_2
0,201907,118785515,CONTINUA,1,1,1
1,201908,118785515,CONTINUA,1,1,1
2,201909,118785515,CONTINUA,1,1,1
3,201910,118785515,CONTINUA,1,1,1
4,201911,118785515,CONTINUA,1,1,1
5,201912,118785515,CONTINUA,1,1,1
6,202001,118785515,CONTINUA,1,1,1
7,202002,118785515,CONTINUA,1,1,1
8,202003,118785515,CONTINUA,1,1,1
9,202004,118785515,CONTINUA,1,1,1


In [137]:
%%time
%%sql
alter table competencia_02 add column clase_ternaria VARCHAR(10)

CPU times: user 157 ms, sys: 36.1 ms, total: 193 ms
Wall time: 372 ms


,Success


In [138]:
%%time
%sql SELECT * FROM COMPETENCIA_02 LIMIT 15

CPU times: user 218 ms, sys: 59.4 ms, total: 277 ms
Wall time: 310 ms


,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,8.0,0.00,-35773.32,2.33,5148.0,36602.13,54.0,0.0,5008.71,None
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,...,1.0,0.00,-67414.82,1149.57,3299.0,28503.06,10.0,0.0,1395.87,None
2,29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,...,1.0,0.00,0.00,0.00,6790.0,5170.25,2.0,0.0,0.00,None
3,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,22.0,69653.59,-59422.00,0.00,7037.0,67991.90,31.0,0.0,2897.31,None
4,29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,...,8.0,0.00,-4802.21,0.00,7596.0,4851.93,3.0,0.0,492.66,None
5,29188277,201901,1,0,1,65,296,1672.70,16113.26,796.59,...,1.0,0.00,-5791.17,0.00,7827.0,12539.39,6.0,0.0,797.64,None
6,29188490,201901,1,0,1,43,281,-520.87,6116.69,2138.32,...,8.0,9172.86,-29810.71,0.00,7568.0,73132.65,41.0,0.0,9172.86,None
7,29188883,201901,1,0,1,64,296,64862.09,459723.16,988.28,...,8.0,0.00,-1332.76,0.00,7504.0,1102.50,1.0,0.0,58.65,None
8,29189950,201901,1,0,1,56,240,13535.41,41250.39,7158.16,...,1.0,0.00,-20419.25,66.25,3717.0,27701.56,19.0,0.0,1137.81,None
9,29190599,201901,1,0,1,59,296,2244.16,24670.53,113.76,...,22.0,8727.12,-16718.98,37.05,7137.0,1388.73,2.0,0.0,1219.92,None


## Seteo el campo clase_ternaria

In [139]:
%%time
%%sql

update competencia_02
   set clase_ternaria = targets.clase_ternaria
  from targets
 where competencia_02.numero_de_cliente = targets.numero_de_cliente
   and competencia_02.foto_mes = targets.foto_mes

CPU times: user 3.43 s, sys: 516 ms, total: 3.95 s
Wall time: 2.17 s


,Success


## Validaciones

In [140]:
%%sql
select numero_de_cliente, foto_mes, clase_ternaria from competencia_02 
--where numero_de_cliente = 29187499
where numero_de_cliente = 29185587
--where numero_de_cliente = 105799389
--where numero_de_cliente = 118785515
 

,numero_de_cliente,foto_mes,clase_ternaria
0,29185587,201901,CONTINUA
1,29185587,201902,CONTINUA
2,29185587,201903,CONTINUA
3,29185587,201904,CONTINUA
4,29185587,201905,CONTINUA
5,29185587,201906,CONTINUA
6,29185587,201907,CONTINUA
7,29185587,201908,CONTINUA
8,29185587,201909,CONTINUA
9,29185587,201910,CONTINUA


## Escribo el archivo

In [141]:
%%time
%%sql
copy competencia_02 to '/Users/mkiszkurno/Documents/dmeyf/datasets/competencia_02.csv.gz' (FORMAT CSV, HEADER)

CPU times: user 6min 49s, sys: 10.4 s, total: 6min 59s
Wall time: 2min 51s


,Success


## Valido archivo Completo

In [142]:
%%time
%%sql
create or replace table competencia_02_verificar as
select
    *
from read_csv_auto('/Users/mkiszkurno/Documents/dmeyf/datasets/competencia_02.csv.gz')

CPU times: user 3min 51s, sys: 2min 1s, total: 5min 52s
Wall time: 2min 18s


,Success


In [150]:
%%time
%sql SELECT count(*) FROM competencia_02_verificar

CPU times: user 117 ms, sys: 174 ms, total: 291 ms
Wall time: 664 ms


,count_star()
0,4562810


# Feature Engineering


Ver todos los campos

In [170]:
%%sql
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'competencia_02';

,column_name,data_type
0,numero_de_cliente,BIGINT
1,foto_mes,BIGINT
2,active_quarter,BIGINT
3,cliente_vip,BIGINT
4,internet,BIGINT
...,...,...
150,Visa_mconsumototal,DOUBLE
151,Visa_cconsumos,BIGINT
152,Visa_cadelantosefectivo,BIGINT
153,Visa_mpagominimo,DOUBLE


Calcular el valor del mes anterior de un campo


In [161]:
%%time
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
from competencia_02
limit 10

CPU times: user 4.12 s, sys: 2.23 s, total: 6.35 s
Wall time: 1.33 s


,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter
0,29188883,201901,43,NaN
1,29188883,201902,45,43.0
2,29188883,201903,39,45.0
3,29188883,201904,38,39.0
4,29188883,201905,39,38.0
5,29188883,201906,41,39.0
6,29188883,201907,34,41.0
7,29188883,201908,30,34.0
8,29188883,201909,25,30.0
9,29188883,201910,30,25.0


Calcular el delta de un campo

In [162]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
from competencia_02
limit 10

,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter,delta_1_ctrx_quarter,lag_2_ctrx_quarter
0,29188883,201901,43,NaN,NaN,NaN
1,29188883,201902,45,43.0,2.0,NaN
2,29188883,201903,39,45.0,-6.0,-4.0
3,29188883,201904,38,39.0,-1.0,-7.0
4,29188883,201905,39,38.0,1.0,0.0
5,29188883,201906,41,39.0,2.0,3.0
6,29188883,201907,34,41.0,-7.0,-5.0
7,29188883,201908,30,34.0,-4.0,-11.0
8,29188883,201909,25,30.0,-5.0,-9.0
9,29188883,201910,30,25.0,5.0,0.0


Si necesitamos ya no solo traer un valor del pasado, sino una secuencia de valores, por ejemplo para calcular la media móvil en los últimos 6 meses, se puede hacer fácilmente



In [163]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_ctrx_quarter
from competencia_02
limit 10

,numero_de_cliente,foto_mes,ctrx_quarter,avg_6_ctrx_quarter
0,29188883,201901,43,43.000000
1,29188883,201902,45,44.000000
2,29188883,201903,39,42.333333
3,29188883,201904,38,41.250000
4,29188883,201905,39,40.800000
5,29188883,201906,41,40.833333
6,29188883,201907,34,39.857143
7,29188883,201908,30,38.000000
8,29188883,201909,25,35.142857
9,29188883,201910,30,33.857143


Si embargo puede resultar incómodo escribir constantemente el over partition sobre todo si se buscan aplicar muchas veces para distintas funciones. Para reducir el código se puede usar la siguiente sintaxis




In [165]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over ventana_6 as ctrx_quarter_media_6
  , max(ctrx_quarter) over ventana_6 as ctrx_quarter_max_6
  , min(ctrx_quarter) over ventana_6 as ctrx_quarter_min_6
from competencia_02
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)
limit 10

,numero_de_cliente,foto_mes,ctrx_quarter,ctrx_quarter_media_6,ctrx_quarter_max_6,ctrx_quarter_min_6
0,29188883,201901,43,43.000000,43,43
1,29188883,201902,45,44.000000,45,43
2,29188883,201903,39,42.333333,45,39
3,29188883,201904,38,41.250000,45,38
4,29188883,201905,39,40.800000,45,38
5,29188883,201906,41,40.833333,45,38
6,29188883,201907,34,39.857143,45,34
7,29188883,201908,30,38.000000,45,30
8,29188883,201909,25,35.142857,41,25
9,29188883,201910,30,33.857143,41,25


Un caso más, que ni me voy a molestar en explicar que significa...



In [166]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6
from competencia_02
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)
limit 10

,numero_de_cliente,foto_mes,ctrx_quarter,ctrx_quarter_slope_6
0,29188883,201901,43,NaN
1,29188883,201902,45,2.000000
2,29188883,201903,39,-2.000000
3,29188883,201904,38,-2.100000
4,29188883,201905,39,-1.500000
5,29188883,201906,41,-0.828571
6,29188883,201907,34,-1.250000
7,29188883,201908,30,-1.857143
8,29188883,201909,25,-2.250000
9,29188883,201910,30,-2.250000


In [172]:
#campos = ['active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 'minversion1_pesos', 'minversion1_dolares', 'cinversion2', 'minversion2', 'cseguro_vida', 'cseguro_auto', 'cseguro_vivienda', 'cseguro_accidentes_personales', 'ccaja_seguridad', 'cpayroll_trx', 'mpayroll', 'mpayroll2', 'cpayroll2_trx', 'ccuenta_debitos_automaticos', 'mcuenta_debitos_automaticos', 'ctarjeta_visa_debitos_automaticos', 'mttarjeta_visa_debitos_automaticos', 'ctarjeta_master_debitos_automaticos', 'mttarjeta_master_debitos_automaticos', 'cpagodeservicios', 'mpagodeservicios', 'cpagomiscuentas', 'mpagomiscuentas', 'ccajeros_propios_descuentos', 'mcajeros_propios_descuentos', 'ctarjeta_visa_descuentos', 'mtarjeta_visa_descuentos', 'ctarjeta_master_descuentos', 'mtarjeta_master_descuentos', 'ccomisiones_mantenimiento', 'mcomisiones_mantenimiento', 'ccomisiones_otras', 'mcomisiones_otras', 'cforex', 'cforex_buy', 'mforex_buy', 'cforex_sell', 'mforex_sell', 'ctransferencias_recibidas', 'mtransferencias_recibidas', 'ctransferencias_emitidas', 'mtransferencias_emitidas', 'cextraccion_autoservicio', 'mextraccion_autoservicio', 'ccheques_depositados', 'mcheques_depositados', 'ccheques_emitidos', 'mcheques_emitidos', 'ccheques_depositados_rechazados', 'mcheques_depositados_rechazados', 'ccheques_emitidos_rechazados', 'mcheques_emitidos_rechazados', 'tcallcenter', 'ccallcenter_transacciones', 'thomebanking', 'chomebanking_transacciones', 'ccajas_transacciones', 'ccajas_consultas', 'ccajas_depositos', 'ccajas_extracciones', 'ccajas_otras', 'catm_trx', 'matm', 'catm_trx_other', 'matm_other', 'ctrx_quarter', 'Master_delinquency', 'Master_status', 'Master_mfinanciacion_limite', 'Master_Fvencimiento', 'Master_Finiciomora', 'Master_msaldototal', 'Master_msaldopesos', 'Master_msaldodolares', 'Master_mconsumospesos', 'Master_mconsumosdolares', 'Master_mlimitecompra', 'Master_madelantopesos', 'Master_madelantodolares', 'Master_fultimo_cierre', 'Master_mpagado', 'Master_mpagospesos', 'Master_mpagosdolares', 'Master_fechaalta', 'Master_mconsumototal', 'Master_cconsumos', 'Master_cadelantosefectivo', 'Master_mpagominimo', 'Visa_delinquency', 'Visa_status', 'Visa_mfinanciacion_limite', 'Visa_Fvencimiento', 'Visa_Finiciomora', 'Visa_msaldototal', 'Visa_msaldopesos', 'Visa_msaldodolares', 'Visa_mconsumospesos', 'Visa_mconsumosdolares', 'Visa_mlimitecompra', 'Visa_madelantopesos', 'Visa_madelantodolares', 'Visa_fultimo_cierre', 'Visa_mpagado', 'Visa_mpagospesos', 'Visa_mpagosdolares', 'Visa_fechaalta', 'Visa_mconsumototal', 'Visa_cconsumos', 'Visa_cadelantosefectivo', 'Visa_mpagominimo']
campos = ['ctrx_quarter','mcaja_ahorro','mcuentas_saldo','cliente_edad','mrentabilidad_annual','numero_de_cliente','cpayroll_trx','mpasivos_margen','mprestamos_personales','mcuenta_corriente','cliente_antiguedad']
print(campos)

['ctrx_quarter', 'mcaja_ahorro', 'mcuentas_saldo', 'cliente_edad', 'mrentabilidad_annual', 'numero_de_cliente', 'cpayroll_trx', 'mpasivos_margen', 'mprestamos_personales', 'mcuenta_corriente', 'cliente_antiguedad']


In [176]:
len(campos)
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_6 as {campo}_slope_6"
  nuevos_features += f", lag({campo}, 1) over ventana_6 as {campo}_lag_1"
  nuevos_features += f", lag({campo}, 2) over ventana_6 as {campo}_lag_2"
  nuevos_features += f", lag({campo}, 3) over ventana_6 as {campo}_lag_3"
  nuevos_features += f", lag({campo}, 4) over ventana_6 as {campo}_lag_4"
  nuevos_features += f", lag({campo}, 5) over ventana_6 as {campo}_lag_5"
  nuevos_features += f", lag({campo}, 6) over ventana_6 as {campo}_lag_6"
  nuevos_features += f", {campo} - lag({campo}, 1) over ventana_6 as {campo}_delta_1_lag"
  nuevos_features += f", {campo} - lag({campo}, 2) over ventana_6 as {campo}_delta_2_lag"
  nuevos_features += f", {campo} - lag({campo}, 3) over ventana_6 as {campo}_delta_3_lag"
  nuevos_features += f", {campo} - lag({campo}, 4) over ventana_6 as {campo}_delta_4_lag"
  nuevos_features += f", {campo} - lag({campo}, 5) over ventana_6 as {campo}_delta_5_lag"
  nuevos_features += f", {campo} - lag({campo}, 6) over ventana_6 as {campo}_delta_6_lag"
  nuevos_features += f", avg({campo}) over ventana_6 as {campo}_media_6"
  nuevos_features += f", max({campo}) over ventana_6 as {campo}_max_6"
  nuevos_features += f",  min({campo}) over ventana_6 as {campo}_min_6"

print(nuevos_features)


, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6, lag(ctrx_quarter, 1) over ventana_6 as ctrx_quarter_lag_1, lag(ctrx_quarter, 2) over ventana_6 as ctrx_quarter_lag_2, lag(ctrx_quarter, 3) over ventana_6 as ctrx_quarter_lag_3, lag(ctrx_quarter, 4) over ventana_6 as ctrx_quarter_lag_4, lag(ctrx_quarter, 5) over ventana_6 as ctrx_quarter_lag_5, lag(ctrx_quarter, 6) over ventana_6 as ctrx_quarter_lag_6, ctrx_quarter - lag(ctrx_quarter, 1) over ventana_6 as ctrx_quarter_delta_1_lag, ctrx_quarter - lag(ctrx_quarter, 2) over ventana_6 as ctrx_quarter_delta_2_lag, ctrx_quarter - lag(ctrx_quarter, 3) over ventana_6 as ctrx_quarter_delta_3_lag, ctrx_quarter - lag(ctrx_quarter, 4) over ventana_6 as ctrx_quarter_delta_4_lag, ctrx_quarter - lag(ctrx_quarter, 5) over ventana_6 as ctrx_quarter_delta_5_lag, ctrx_quarter - lag(ctrx_quarter, 6) over ventana_6 as ctrx_quarter_delta_6_lag, avg(ctrx_quarter) over ventana_6 as ctrx_quarter_media_6, max(ctrx_quarter) ov

In [178]:
sql_query = f"""
SELECT
  *
  {nuevos_features}
FROM competencia_02
WINDOW ventana_6 AS (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)
"""
print (sql_query)


SELECT
  numero_de_cliente,
  foto_mes,
  ctrx_quarter,
  regr_slope(ctrx_quarter, cliente_antiguedad) OVER ventana_6 AS ctrx_quarter_slope_6,
  
, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6, lag(ctrx_quarter, 1) over ventana_6 as ctrx_quarter_lag_1, lag(ctrx_quarter, 2) over ventana_6 as ctrx_quarter_lag_2, lag(ctrx_quarter, 3) over ventana_6 as ctrx_quarter_lag_3, lag(ctrx_quarter, 4) over ventana_6 as ctrx_quarter_lag_4, lag(ctrx_quarter, 5) over ventana_6 as ctrx_quarter_lag_5, lag(ctrx_quarter, 6) over ventana_6 as ctrx_quarter_lag_6, ctrx_quarter - lag(ctrx_quarter, 1) over ventana_6 as ctrx_quarter_delta_1_lag, ctrx_quarter - lag(ctrx_quarter, 2) over ventana_6 as ctrx_quarter_delta_2_lag, ctrx_quarter - lag(ctrx_quarter, 3) over ventana_6 as ctrx_quarter_delta_3_lag, ctrx_quarter - lag(ctrx_quarter, 4) over ventana_6 as ctrx_quarter_delta_4_lag, ctrx_quarter - lag(ctrx_quarter, 5) over ventana_6 as ctrx_quarter_delta_5_lag, ctrx_quarter

In [182]:
%%time
%%sql
create or replace table competencia_02 as
SELECT
  *, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6, lag(ctrx_quarter, 1) over ventana_6 as ctrx_quarter_lag_1, lag(ctrx_quarter, 2) over ventana_6 as ctrx_quarter_lag_2, lag(ctrx_quarter, 3) over ventana_6 as ctrx_quarter_lag_3, lag(ctrx_quarter, 4) over ventana_6 as ctrx_quarter_lag_4, lag(ctrx_quarter, 5) over ventana_6 as ctrx_quarter_lag_5, lag(ctrx_quarter, 6) over ventana_6 as ctrx_quarter_lag_6, ctrx_quarter - lag(ctrx_quarter, 1) over ventana_6 as ctrx_quarter_delta_1_lag, ctrx_quarter - lag(ctrx_quarter, 2) over ventana_6 as ctrx_quarter_delta_2_lag, ctrx_quarter - lag(ctrx_quarter, 3) over ventana_6 as ctrx_quarter_delta_3_lag, ctrx_quarter - lag(ctrx_quarter, 4) over ventana_6 as ctrx_quarter_delta_4_lag, ctrx_quarter - lag(ctrx_quarter, 5) over ventana_6 as ctrx_quarter_delta_5_lag, ctrx_quarter - lag(ctrx_quarter, 6) over ventana_6 as ctrx_quarter_delta_6_lag, avg(ctrx_quarter) over ventana_6 as ctrx_quarter_media_6, max(ctrx_quarter) over ventana_6 as ctrx_quarter_max_6,  min(ctrx_quarter) over ventana_6 as ctrx_quarter_min_6
, regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_6 as mcaja_ahorro_slope_6, lag(mcaja_ahorro, 1) over ventana_6 as mcaja_ahorro_lag_1, lag(mcaja_ahorro, 2) over ventana_6 as mcaja_ahorro_lag_2, lag(mcaja_ahorro, 3) over ventana_6 as mcaja_ahorro_lag_3, lag(mcaja_ahorro, 4) over ventana_6 as mcaja_ahorro_lag_4, lag(mcaja_ahorro, 5) over ventana_6 as mcaja_ahorro_lag_5, lag(mcaja_ahorro, 6) over ventana_6 as mcaja_ahorro_lag_6, mcaja_ahorro - lag(mcaja_ahorro, 1) over ventana_6 as mcaja_ahorro_delta_1_lag, mcaja_ahorro - lag(mcaja_ahorro, 2) over ventana_6 as mcaja_ahorro_delta_2_lag, mcaja_ahorro - lag(mcaja_ahorro, 3) over ventana_6 as mcaja_ahorro_delta_3_lag, mcaja_ahorro - lag(mcaja_ahorro, 4) over ventana_6 as mcaja_ahorro_delta_4_lag, mcaja_ahorro - lag(mcaja_ahorro, 5) over ventana_6 as mcaja_ahorro_delta_5_lag, mcaja_ahorro - lag(mcaja_ahorro, 6) over ventana_6 as mcaja_ahorro_delta_6_lag, avg(mcaja_ahorro) over ventana_6 as mcaja_ahorro_media_6, max(mcaja_ahorro) over ventana_6 as mcaja_ahorro_max_6,  min(mcaja_ahorro) over ventana_6 as mcaja_ahorro_min_6
, regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_6 as mcuentas_saldo_slope_6, lag(mcuentas_saldo, 1) over ventana_6 as mcuentas_saldo_lag_1, lag(mcuentas_saldo, 2) over ventana_6 as mcuentas_saldo_lag_2, lag(mcuentas_saldo, 3) over ventana_6 as mcuentas_saldo_lag_3, lag(mcuentas_saldo, 4) over ventana_6 as mcuentas_saldo_lag_4, lag(mcuentas_saldo, 5) over ventana_6 as mcuentas_saldo_lag_5, lag(mcuentas_saldo, 6) over ventana_6 as mcuentas_saldo_lag_6, mcuentas_saldo - lag(mcuentas_saldo, 1) over ventana_6 as mcuentas_saldo_delta_1_lag, mcuentas_saldo - lag(mcuentas_saldo, 2) over ventana_6 as mcuentas_saldo_delta_2_lag, mcuentas_saldo - lag(mcuentas_saldo, 3) over ventana_6 as mcuentas_saldo_delta_3_lag, mcuentas_saldo - lag(mcuentas_saldo, 4) over ventana_6 as mcuentas_saldo_delta_4_lag, mcuentas_saldo - lag(mcuentas_saldo, 5) over ventana_6 as mcuentas_saldo_delta_5_lag, mcuentas_saldo - lag(mcuentas_saldo, 6) over ventana_6 as mcuentas_saldo_delta_6_lag, avg(mcuentas_saldo) over ventana_6 as mcuentas_saldo_media_6, max(mcuentas_saldo) over ventana_6 as mcuentas_saldo_max_6,  min(mcuentas_saldo) over ventana_6 as mcuentas_saldo_min_6
, regr_slope(cliente_edad, cliente_antiguedad) over ventana_6 as cliente_edad_slope_6, lag(cliente_edad, 1) over ventana_6 as cliente_edad_lag_1, lag(cliente_edad, 2) over ventana_6 as cliente_edad_lag_2, lag(cliente_edad, 3) over ventana_6 as cliente_edad_lag_3, lag(cliente_edad, 4) over ventana_6 as cliente_edad_lag_4, lag(cliente_edad, 5) over ventana_6 as cliente_edad_lag_5, lag(cliente_edad, 6) over ventana_6 as cliente_edad_lag_6, cliente_edad - lag(cliente_edad, 1) over ventana_6 as cliente_edad_delta_1_lag, cliente_edad - lag(cliente_edad, 2) over ventana_6 as cliente_edad_delta_2_lag, cliente_edad - lag(cliente_edad, 3) over ventana_6 as cliente_edad_delta_3_lag, cliente_edad - lag(cliente_edad, 4) over ventana_6 as cliente_edad_delta_4_lag, cliente_edad - lag(cliente_edad, 5) over ventana_6 as cliente_edad_delta_5_lag, cliente_edad - lag(cliente_edad, 6) over ventana_6 as cliente_edad_delta_6_lag, avg(cliente_edad) over ventana_6 as cliente_edad_media_6, max(cliente_edad) over ventana_6 as cliente_edad_max_6,  min(cliente_edad) over ventana_6 as cliente_edad_min_6
, regr_slope(mrentabilidad_annual, cliente_antiguedad) over ventana_6 as mrentabilidad_annual_slope_6, lag(mrentabilidad_annual, 1) over ventana_6 as mrentabilidad_annual_lag_1, lag(mrentabilidad_annual, 2) over ventana_6 as mrentabilidad_annual_lag_2, lag(mrentabilidad_annual, 3) over ventana_6 as mrentabilidad_annual_lag_3, lag(mrentabilidad_annual, 4) over ventana_6 as mrentabilidad_annual_lag_4, lag(mrentabilidad_annual, 5) over ventana_6 as mrentabilidad_annual_lag_5, lag(mrentabilidad_annual, 6) over ventana_6 as mrentabilidad_annual_lag_6, mrentabilidad_annual - lag(mrentabilidad_annual, 1) over ventana_6 as mrentabilidad_annual_delta_1_lag, mrentabilidad_annual - lag(mrentabilidad_annual, 2) over ventana_6 as mrentabilidad_annual_delta_2_lag, mrentabilidad_annual - lag(mrentabilidad_annual, 3) over ventana_6 as mrentabilidad_annual_delta_3_lag, mrentabilidad_annual - lag(mrentabilidad_annual, 4) over ventana_6 as mrentabilidad_annual_delta_4_lag, mrentabilidad_annual - lag(mrentabilidad_annual, 5) over ventana_6 as mrentabilidad_annual_delta_5_lag, mrentabilidad_annual - lag(mrentabilidad_annual, 6) over ventana_6 as mrentabilidad_annual_delta_6_lag, avg(mrentabilidad_annual) over ventana_6 as mrentabilidad_annual_media_6, max(mrentabilidad_annual) over ventana_6 as mrentabilidad_annual_max_6,  min(mrentabilidad_annual) over ventana_6 as mrentabilidad_annual_min_6
, regr_slope(numero_de_cliente, cliente_antiguedad) over ventana_6 as numero_de_cliente_slope_6, lag(numero_de_cliente, 1) over ventana_6 as numero_de_cliente_lag_1, lag(numero_de_cliente, 2) over ventana_6 as numero_de_cliente_lag_2, lag(numero_de_cliente, 3) over ventana_6 as numero_de_cliente_lag_3, lag(numero_de_cliente, 4) over ventana_6 as numero_de_cliente_lag_4, lag(numero_de_cliente, 5) over ventana_6 as numero_de_cliente_lag_5, lag(numero_de_cliente, 6) over ventana_6 as numero_de_cliente_lag_6, numero_de_cliente - lag(numero_de_cliente, 1) over ventana_6 as numero_de_cliente_delta_1_lag, numero_de_cliente - lag(numero_de_cliente, 2) over ventana_6 as numero_de_cliente_delta_2_lag, numero_de_cliente - lag(numero_de_cliente, 3) over ventana_6 as numero_de_cliente_delta_3_lag, numero_de_cliente - lag(numero_de_cliente, 4) over ventana_6 as numero_de_cliente_delta_4_lag, numero_de_cliente - lag(numero_de_cliente, 5) over ventana_6 as numero_de_cliente_delta_5_lag, numero_de_cliente - lag(numero_de_cliente, 6) over ventana_6 as numero_de_cliente_delta_6_lag, avg(numero_de_cliente) over ventana_6 as numero_de_cliente_media_6, max(numero_de_cliente) over ventana_6 as numero_de_cliente_max_6,  min(numero_de_cliente) over ventana_6 as numero_de_cliente_min_6
, regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_6 as cpayroll_trx_slope_6, lag(cpayroll_trx, 1) over ventana_6 as cpayroll_trx_lag_1, lag(cpayroll_trx, 2) over ventana_6 as cpayroll_trx_lag_2, lag(cpayroll_trx, 3) over ventana_6 as cpayroll_trx_lag_3, lag(cpayroll_trx, 4) over ventana_6 as cpayroll_trx_lag_4, lag(cpayroll_trx, 5) over ventana_6 as cpayroll_trx_lag_5, lag(cpayroll_trx, 6) over ventana_6 as cpayroll_trx_lag_6, cpayroll_trx - lag(cpayroll_trx, 1) over ventana_6 as cpayroll_trx_delta_1_lag, cpayroll_trx - lag(cpayroll_trx, 2) over ventana_6 as cpayroll_trx_delta_2_lag, cpayroll_trx - lag(cpayroll_trx, 3) over ventana_6 as cpayroll_trx_delta_3_lag, cpayroll_trx - lag(cpayroll_trx, 4) over ventana_6 as cpayroll_trx_delta_4_lag, cpayroll_trx - lag(cpayroll_trx, 5) over ventana_6 as cpayroll_trx_delta_5_lag, cpayroll_trx - lag(cpayroll_trx, 6) over ventana_6 as cpayroll_trx_delta_6_lag, avg(cpayroll_trx) over ventana_6 as cpayroll_trx_media_6, max(cpayroll_trx) over ventana_6 as cpayroll_trx_max_6,  min(cpayroll_trx) over ventana_6 as cpayroll_trx_min_6
, regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_6 as mpasivos_margen_slope_6, lag(mpasivos_margen, 1) over ventana_6 as mpasivos_margen_lag_1, lag(mpasivos_margen, 2) over ventana_6 as mpasivos_margen_lag_2, lag(mpasivos_margen, 3) over ventana_6 as mpasivos_margen_lag_3, lag(mpasivos_margen, 4) over ventana_6 as mpasivos_margen_lag_4, lag(mpasivos_margen, 5) over ventana_6 as mpasivos_margen_lag_5, lag(mpasivos_margen, 6) over ventana_6 as mpasivos_margen_lag_6, mpasivos_margen - lag(mpasivos_margen, 1) over ventana_6 as mpasivos_margen_delta_1_lag, mpasivos_margen - lag(mpasivos_margen, 2) over ventana_6 as mpasivos_margen_delta_2_lag, mpasivos_margen - lag(mpasivos_margen, 3) over ventana_6 as mpasivos_margen_delta_3_lag, mpasivos_margen - lag(mpasivos_margen, 4) over ventana_6 as mpasivos_margen_delta_4_lag, mpasivos_margen - lag(mpasivos_margen, 5) over ventana_6 as mpasivos_margen_delta_5_lag, mpasivos_margen - lag(mpasivos_margen, 6) over ventana_6 as mpasivos_margen_delta_6_lag, avg(mpasivos_margen) over ventana_6 as mpasivos_margen_media_6, max(mpasivos_margen) over ventana_6 as mpasivos_margen_max_6,  min(mpasivos_margen) over ventana_6 as mpasivos_margen_min_6
, regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_6 as mprestamos_personales_slope_6, lag(mprestamos_personales, 1) over ventana_6 as mprestamos_personales_lag_1, lag(mprestamos_personales, 2) over ventana_6 as mprestamos_personales_lag_2, lag(mprestamos_personales, 3) over ventana_6 as mprestamos_personales_lag_3, lag(mprestamos_personales, 4) over ventana_6 as mprestamos_personales_lag_4, lag(mprestamos_personales, 5) over ventana_6 as mprestamos_personales_lag_5, lag(mprestamos_personales, 6) over ventana_6 as mprestamos_personales_lag_6, mprestamos_personales - lag(mprestamos_personales, 1) over ventana_6 as mprestamos_personales_delta_1_lag, mprestamos_personales - lag(mprestamos_personales, 2) over ventana_6 as mprestamos_personales_delta_2_lag, mprestamos_personales - lag(mprestamos_personales, 3) over ventana_6 as mprestamos_personales_delta_3_lag, mprestamos_personales - lag(mprestamos_personales, 4) over ventana_6 as mprestamos_personales_delta_4_lag, mprestamos_personales - lag(mprestamos_personales, 5) over ventana_6 as mprestamos_personales_delta_5_lag, mprestamos_personales - lag(mprestamos_personales, 6) over ventana_6 as mprestamos_personales_delta_6_lag, avg(mprestamos_personales) over ventana_6 as mprestamos_personales_media_6, max(mprestamos_personales) over ventana_6 as mprestamos_personales_max_6,  min(mprestamos_personales) over ventana_6 as mprestamos_personales_min_6
, regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_6 as mcuenta_corriente_slope_6, lag(mcuenta_corriente, 1) over ventana_6 as mcuenta_corriente_lag_1, lag(mcuenta_corriente, 2) over ventana_6 as mcuenta_corriente_lag_2, lag(mcuenta_corriente, 3) over ventana_6 as mcuenta_corriente_lag_3, lag(mcuenta_corriente, 4) over ventana_6 as mcuenta_corriente_lag_4, lag(mcuenta_corriente, 5) over ventana_6 as mcuenta_corriente_lag_5, lag(mcuenta_corriente, 6) over ventana_6 as mcuenta_corriente_lag_6, mcuenta_corriente - lag(mcuenta_corriente, 1) over ventana_6 as mcuenta_corriente_delta_1_lag, mcuenta_corriente - lag(mcuenta_corriente, 2) over ventana_6 as mcuenta_corriente_delta_2_lag, mcuenta_corriente - lag(mcuenta_corriente, 3) over ventana_6 as mcuenta_corriente_delta_3_lag, mcuenta_corriente - lag(mcuenta_corriente, 4) over ventana_6 as mcuenta_corriente_delta_4_lag, mcuenta_corriente - lag(mcuenta_corriente, 5) over ventana_6 as mcuenta_corriente_delta_5_lag, mcuenta_corriente - lag(mcuenta_corriente, 6) over ventana_6 as mcuenta_corriente_delta_6_lag, avg(mcuenta_corriente) over ventana_6 as mcuenta_corriente_media_6, max(mcuenta_corriente) over ventana_6 as mcuenta_corriente_max_6,  min(mcuenta_corriente) over ventana_6 as mcuenta_corriente_min_6
, regr_slope(cliente_antiguedad, cliente_antiguedad) over ventana_6 as cliente_antiguedad_slope_6, lag(cliente_antiguedad, 1) over ventana_6 as cliente_antiguedad_lag_1, lag(cliente_antiguedad, 2) over ventana_6 as cliente_antiguedad_lag_2, lag(cliente_antiguedad, 3) over ventana_6 as cliente_antiguedad_lag_3, lag(cliente_antiguedad, 4) over ventana_6 as cliente_antiguedad_lag_4, lag(cliente_antiguedad, 5) over ventana_6 as cliente_antiguedad_lag_5, lag(cliente_antiguedad, 6) over ventana_6 as cliente_antiguedad_lag_6, cliente_antiguedad - lag(cliente_antiguedad, 1) over ventana_6 as cliente_antiguedad_delta_1_lag, cliente_antiguedad - lag(cliente_antiguedad, 2) over ventana_6 as cliente_antiguedad_delta_2_lag, cliente_antiguedad - lag(cliente_antiguedad, 3) over ventana_6 as cliente_antiguedad_delta_3_lag, cliente_antiguedad - lag(cliente_antiguedad, 4) over ventana_6 as cliente_antiguedad_delta_4_lag, cliente_antiguedad - lag(cliente_antiguedad, 5) over ventana_6 as cliente_antiguedad_delta_5_lag, cliente_antiguedad - lag(cliente_antiguedad, 6) over ventana_6 as cliente_antiguedad_delta_6_lag, avg(cliente_antiguedad) over ventana_6 as cliente_antiguedad_media_6, max(cliente_antiguedad) over ventana_6 as cliente_antiguedad_max_6,  min(cliente_antiguedad) over ventana_6 as cliente_antiguedad_min_6
FROM competencia_02
WINDOW ventana_6 AS (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)


CPU times: user 5min 55s, sys: 3min 1s, total: 8min 56s
Wall time: 1min 35s


,Success


## Escribo el archivo

In [183]:
%%time
%%sql
copy competencia_02 to '/Users/mkiszkurno/Documents/dmeyf/datasets/competencia_02.csv.gz' (FORMAT CSV, HEADER)